# FHV trip data

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
from jupyterthemes import jtplot
jtplot.style()
%matplotlib inline
plt.style.use('fivethirtyeight')

In [2]:
data = 'C:/Users/cwnos/Documents/DSC630/Final Project/fhvhv_tripdata_2020-02.csv'

In [3]:
TextFileReader = pd.read_csv(data, chunksize=500000)  # the number of rows per chunk

dfList = []
for df in TextFileReader:
    dfList.append(df)

df = pd.concat(dfList,sort=False)

In [4]:
df = pd.read_csv(data, nrows=500000)  # the number of rows per chunk

In [5]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y/%m/%d')
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'], format='%Y/%m/%d')


In [6]:
df['Duration (seconds)'] = df['dropoff_datetime'] - df['pickup_datetime']

df['Duration (seconds)'] = df['Duration (seconds)'] / np.timedelta64(1,'s')


In [9]:
print(df['Duration (seconds)'].median())
print(df['Duration (seconds)'].mean())

881.0
1063.3776499876985


In [10]:
df['year'] = df['pickup_datetime'].dt.year
df['month'] = df['pickup_datetime'].dt.month
df['day'] = df['pickup_datetime'].dt.day
df['day_of_week'] = df['pickup_datetime'].dt.dayofweek
df['hour'] = df['pickup_datetime'].dt.hour
df['minute'] = df['pickup_datetime'].dt.minute

In [11]:
day_mapping = {0:'Monday',
              1:'Tuesday',
              2:'Wednesday',
              3:'Thursday',
              4:'Friday',
              5:'Saturday',
              6:'Sunday'}

df['day_name'] = df['pickup_datetime'].dt.weekday.map(day_mapping)

In [12]:
provider_mapping = {'HV0003': 'Uber',
                   'HV0005': 'Lyft',
                   'HV0002': 'Juno',
                   'HV0004': 'Via'}

df['provider'] = df['hvfhs_license_num'].map(provider_mapping)

In [13]:
#df.head()
df['avg_seconds'] = df['Duration (seconds)'].median()

In [14]:
#df.shape
df = df.drop(['hvfhs_license_num', 'dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'SR_Flag'], axis = 1)

In [15]:
#fig = plt.figure(figsize = (10,10))
#plt.hist(dfUber['day_name'], bins = 28)
#plt.xlabel('Day of week')
#plt.ylabel('Count')
#plt.xticks(rotation = 45)
#plt.show()

In [16]:
dfSub = df.sample(n=5000000,random_state=42)

In [17]:
# Subsetting DF to exclude cases where rides exceed 28800 seconds (8 hours) 10 total records. Can lower threshold to 21600 (6 hours) and remove an additional 10 records.
# Subset further to exclude rides less than 2500 seconds
df = dfSub[dfSub['Duration (seconds)'] >= 1] 
#df = df[df['Duration (seconds)']< 30000] 


In [18]:
#fig = plt.figure(figsize = (10,30))

#plt.boxplot(dfUber_trunc['Duration (seconds)'])
#plt.ylabel('Duration')

#plt.show()


In [19]:
#fig = plt.figure(figsize = (30,10))

#plt.hist(dfUber_trunc['Duration (seconds)'], bins = 1500)
#plt.xlabel('Duration')
#plt.ylabel('Count')
#plt.title('Histogram Duration (rides less than 2100 seconds)')
#plt.show()


In [20]:
def normalize(column):
    upper = column.max()
    lower = column.min()
    y = (column) / (upper)
    return y

#duration_normalized = normalize(dfUber_trunc['Duration (seconds)'])
#duration_normalized.describe()

#dfUber_trunc['duration_normalized'] = duration_normalized

#dfUber_trunc.head()

In [21]:
#fig = plt.figure(figsize = (30,10))

#plt.hist(dfUber_trunc.duration_normalized, bins = 1500)
#plt.xlabel('Duration')
#plt.ylabel('Count')
#plt.title('Histogram Duration (rides less than 2100 seconds)')
#plt.show()

In [22]:
#dfUber_trunc.describe()
df.describe()

,PULocationID,DOLocationID,Duration (seconds),year,month,day,day_of_week,hour,minute,avg_seconds
count,4.999950e+06,4.999950e+06,4.999950e+06,4999950.0,4999950.0,4.999950e+06,4.999950e+06,4.999950e+06,4.999950e+06,4999950.0
mean,1.393969e+02,1.419726e+02,1.063302e+03,2020.0,2.0,1.517707e+01,3.282088e+00,1.363397e+01,2.947024e+01,881.0
std,7.511630e+01,7.753659e+01,7.269379e+02,0.0,0.0,8.485060e+00,1.948590e+00,6.477484e+00,1.732431e+01,0.0
min,1.000000e+00,1.000000e+00,1.000000e+00,2020.0,2.0,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,881.0
25%,7.600000e+01,7.600000e+01,5.570000e+02,2020.0,2.0,8.000000e+00,2.000000e+00,9.000000e+00,1.400000e+01,881.0
50%,1.410000e+02,1.420000e+02,8.810000e+02,2020.0,2.0,1.500000e+01,4.000000e+00,1.500000e+01,3.000000e+01,881.0
75%,2.110000e+02,2.180000e+02,1.368000e+03,2020.0,2.0,2.200000e+01,5.000000e+00,1.900000e+01,4.400000e+01,881.0
max,2.650000e+02,2.650000e+02,8.489400e+04,2020.0,2.0,2.900000e+01,6.000000e+00,2.300000e+01,5.900000e+01,881.0


In [23]:
df = pd.get_dummies(df)

#df.iloc[:,5:].head
df

,PULocationID,DOLocationID,Duration (seconds),year,month,day,day_of_week,hour,minute,avg_seconds,day_name_Friday,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday,provider_Lyft,provider_Uber,provider_Via
10999287,164,246,503.0,2020,2,15,5,18,26,881.0,0,0,1,0,0,0,0,0,1,0
20265306,148,114,366.0,2020,2,28,4,13,6,881.0,1,0,0,0,0,0,0,0,1,0
11933773,159,254,1287.0,2020,2,16,6,20,19,881.0,0,0,0,1,0,0,0,0,1,0
19289582,181,144,1796.0,2020,2,27,3,9,55,881.0,0,0,0,0,1,0,0,0,1,0
3021005,52,144,1280.0,2020,2,5,2,9,44,881.0,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14308364,138,233,2131.0,2020,2,20,3,17,32,881.0,0,0,0,0,1,0,0,1,0,0
13368241,139,218,372.0,2020,2,19,2,8,58,881.0,0,0,0,0,0,0,1,0,1,0
5893538,231,231,233.0,2020,2,8,5,21,39,881.0,0,0,1,0,0,0,0,0,1,0
14736260,50,170,821.0,2020,2,21,4,7,45,881.0,1,0,0,0,0,0,0,1,0,0


In [24]:
# labels = value we want to predict
labels = np.array(df['Duration (seconds)'])

#remove labels from set
df = df.drop('Duration (seconds)', axis = 1)

df_list = list(df.columns)

df = np.array(df)

In [25]:
from sklearn.model_selection import train_test_split as split

In [26]:
train_df, test_df, train_labels, test_labels = split(df, labels, test_size = 0.25, random_state = 42)

In [27]:
print('Training df Shape:', train_df.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing df Shape:', test_df.shape)
print('Testing Labels Shape:', test_labels.shape)

Training df Shape: (3749962, 19)
Training Labels Shape: (3749962,)
Testing df Shape: (1249988, 19)
Testing Labels Shape: (1249988,)


In [28]:
# The baseline predictions are the historical averages
baseline_preds = test_df[:, df_list.index('avg_seconds')]
# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

Average baseline error:  510.71


In [29]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 3, random_state = 42)
# Train the model on training data
rf.fit(train_df, train_labels);

In [34]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_df)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.median(errors), 2), 'seconds.')

print(errors.min())

Mean Absolute Error: 217.67 seconds.
0.0


In [36]:
# Calculate mean absolute percentage error (MAPE)
mape =  100 * (errors / test_labels) 
# Calculate and display accuracy
accuracy = 100 - np.median(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 74.99 %.
